In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_char_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [2]:
wiki.name

'wikipedia'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OllamaEmbeddings(
    model="llama2",
)

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001971F025E70>)

In [4]:
from langchain.tools.retriever import create_retriever_tool
lang=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [5]:
lang.name

'langsmith_search'

In [6]:
## Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [7]:
tools = [wiki,arxiv,lang]

In [8]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\mail2\\Desktop\\Debesh\\Chatbot\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000019710188D30>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<lang

In [29]:
from langchain_community.llms import Ollama
import os
## Ollama llama2 LLM
llm=Ollama(model = "llama2")

In [10]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\mail2\Desktop\Debesh\Chatbot\venv\lib\site-packages\langsmith\client.py:5489: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [37]:
## Agents
from langchain.agents import create_openai_tools_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
olla = OllamaFunctions(model="llama2")

model = ChatOpenAI()
agent = create_openai_tools_agent(olla,tools,prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [38]:
## Agent Executor
from langchain_core.runnables.base import Runnable
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [39]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Langsmith is a small village nestled in the heart of the countryside, surrounded by rolling hills and picturesque farmland. It is a peaceful and idyllic place to visit or live, with a strong sense of community and a rich history.

The village has a population of around 500 people, who are known for their warm hospitality and friendly nature. Visitors often comment on the welcoming atmosphere and the feeling of being part of a tight-knit community.

Langsmith has a few local businesses, including a general store, a post office, and a small cafe. These businesses provide essential services to the villagers, as well as catering to the needs of visitors. The village also hosts a few annual events, such as a summer fair and a Christmas market, which bring the community together and attract visitors from nearby towns and cities.

The village is home to several historic landmarks, including a medieval church and a 16th-century manor house. These buildi

{'input': 'Tell me about Langsmith',
 'output': "\nLangsmith is a small village nestled in the heart of the countryside, surrounded by rolling hills and picturesque farmland. It is a peaceful and idyllic place to visit or live, with a strong sense of community and a rich history.\n\nThe village has a population of around 500 people, who are known for their warm hospitality and friendly nature. Visitors often comment on the welcoming atmosphere and the feeling of being part of a tight-knit community.\n\nLangsmith has a few local businesses, including a general store, a post office, and a small cafe. These businesses provide essential services to the villagers, as well as catering to the needs of visitors. The village also hosts a few annual events, such as a summer fair and a Christmas market, which bring the community together and attract visitors from nearby towns and cities.\n\nThe village is home to several historic landmarks, including a medieval church and a 16th-century manor hou

In [40]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Of course! Machine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a specific task over time. The goal of machine learning is to create a system that can learn and improve its performance without being explicitly programmed for each task.

There are several types of machine learning, including:

1. Supervised Learning: In this type of machine learning, the algorithm is trained on labeled data, where the correct output is already known. The algorithm learns to map inputs to outputs based on the labeled data, and can then make predictions on new, unseen data. Examples of supervised learning include image classification, speech recognition, and sentiment analysis.
2. Unsupervised Learning: In this type of machine learning, the algorithm is trained on unlabeled data, and it must find patterns or st

{'input': 'Tell me about Machine Learning',
 'output': "\nOf course! Machine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a specific task over time. The goal of machine learning is to create a system that can learn and improve its performance without being explicitly programmed for each task.\n\nThere are several types of machine learning, including:\n\n1. Supervised Learning: In this type of machine learning, the algorithm is trained on labeled data, where the correct output is already known. The algorithm learns to map inputs to outputs based on the labeled data, and can then make predictions on new, unseen data. Examples of supervised learning include image classification, speech recognition, and sentiment analysis.\n2. Unsupervised Learning: In this type of machine learning, the algorithm is trained on unlabeled data, and it mu

In [41]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

As a helpful assistant, I can provide you with information on various topics, including scientific papers. The paper you mentioned, "1605.08386," is likely a reference to a specific scientific article or research paper.

Unfortunately, I don't have access to the full text of the paper without more context. Can you please provide me with more information about the paper, such as the authors, title, or field of study? That way, I can try to help you find the paper or provide you with additional information about it.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': '\nAs a helpful assistant, I can provide you with information on various topics, including scientific papers. The paper you mentioned, "1605.08386," is likely a reference to a specific scientific article or research paper.\n\nUnfortunately, I don\'t have access to the full text of the paper without more context. Can you please provide me with more information about the paper, such as the authors, title, or field of study? That way, I can try to help you find the paper or provide you with additional information about it.'}